In [26]:
# music_classifier_model_with_outputs.py

# ============================STEP 0: IMPORT LIBRARY============================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Untuk progress bar (opsional tapi bagus untuk data besar)
import datetime # Untuk timestamp di log

def log_step(message):
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

def load_data(file_path):
    """
    Memuat dataset dari file CSV.
    Output: DataFrame awal, menampilkan head dan info dasar.
    """
    log_step(f"STEP 1: Memuat data dari jalur yang diberikan: '{file_path}'...")
    
    # --- DEBUGGING JALUR FILE SECARA EKSPLISIT ---
    # Mendapatkan jalur absolut dari file_path yang diberikan
    # Jika file_path sudah absolut, abspath akan mengembalikannya apa adanya
    # Jika relatif, ia akan menggabungkannya dengan CWD
    absolute_file_path = os.path.abspath(file_path)
    log_step(f"Jalur absolut yang dicoba: '{absolute_file_path}'")
    
    if not os.path.exists(absolute_file_path):
        log_step(f"ERROR: File TIDAK DITEMUKAN di jalur: '{absolute_file_path}'.")
        log_step("Mohon periksa ejaan, casing (huruf besar/kecil), dan lokasi file.")
        log_step("Pastikan Anda menjalankan script dari direktori yang benar atau gunakan jalur absolut yang valid.")
        return None # Mengembalikan None jika file tidak ditemukan
    # --- AKHIR DEBUGGING JALUR FILE ---
    
    try:
        # Gunakan file_path yang dilewatkan sebagai argumen.
        # Jika Anda hardcode path di sini sebelumnya, HAPUS hardcode itu.
        # Contoh: df = pd.read_csv("Progress 6 June 2025/SpotifyFeatures.csv")
        # GANTI KEMBALI menjadi:
        df = pd.read_csv(file_path) 
        
        log_step(f"Data berhasil dimuat. {len(df)} baris ditemukan.")
        log_step("\n--- Preview Data (Head) ---")
        print(df.head())
        log_step("\n--- Informasi Data (Info) ---")
        df.info()
        log_step("\n--- Statistik Deskriptif ---")
        print(df.describe())
        return df
    except pd.errors.EmptyDataError:
        log_step(f"ERROR: File CSV kosong atau tidak memiliki kolom di jalur: '{file_path}'.")
        return None
    except pd.errors.ParserError as pe:
        log_step(f"ERROR: Kesalahan parsing CSV di jalur: '{file_path}'. Detail: {pe}")
        log_step("Mungkin format CSV rusak atau tidak sesuai standar.")
        return None
    except Exception as e:
        log_step(f"ERROR umum saat memuat data dari '{file_path}': {e}")
        log_step("Mungkin ada masalah dengan format file, izin akses, atau memori.")
        return None

In [27]:
# ============================STEP 2: PREPROCESSING DATA============================
def preprocess_data(df):
    """
    Melakukan preprocessing pada data:
    - Memilih fitur numerik yang relevan.
    - Menangani nilai yang hilang.
    - Menghapus duplikat.
    - Normalisasi fitur numerik.
    Output: DataFrame setelah setiap tahap preprocessing.
    """
    log_step("STEP 2: Memulai preprocessing data...")

    initial_rows = len(df)
    log_step(f"Baris awal: {initial_rows}")

    # Definisi fitur numerik yang akan digunakan untuk clustering
    numeric_features = [
        'danceability', 'energy', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'loudness', 'speechiness'
    ]
    
    # Tambahkan 'genre', 'track_name', 'artist_name' untuk referensi dan penggabungan nanti
    non_numeric_features = ['track_name', 'artist_name', 'genre']
    all_relevant_cols = numeric_features + non_numeric_features

    # Pastikan semua fitur yang dibutuhkan ada
    missing_cols = [col for col in all_relevant_cols if col not in df.columns]
    if missing_cols:
        log_step(f"ERROR: Kolom berikut tidak ditemukan di dataset: {missing_cols}")
        log_step("Pastikan nama kolom di dataset sesuai dengan yang didefinisikan.")
        exit()

    # Memilih kolom yang relevan
    df_processed = df[all_relevant_cols].copy()
    log_step(f"Memilih {len(all_relevant_cols)} kolom yang relevan.")
    log_step("\n--- Data Setelah Pemilihan Kolom (Head) ---")
    print(df_processed.head())

    # Menangani nilai yang hilang (jika ada)
    log_step("Memeriksa dan mengisi nilai yang hilang...")
    missing_counts = df_processed[numeric_features].isnull().sum()
    if missing_counts.sum() > 0:
        log_step("Nilai hilang ditemukan di kolom:")
        print(missing_counts[missing_counts > 0])
        for feature in tqdm(numeric_features, desc="Mengisi nilai hilang"):
            if df_processed[feature].isnull().any():
                median_val = df_processed[feature].median()
                df_processed[feature].fillna(median_val, inplace=True)
                log_step(f"  Mengisi nilai hilang di '{feature}' dengan median: {median_val}")
    else:
        log_step("Tidak ada nilai hilang ditemukan di fitur numerik.")

    log_step("\n--- Data Setelah Penanganan Nilai Hilang (Info) ---")
    df_processed.info()

    # Menghapus duplikat
    log_step("Menghapus duplikat berdasarkan 'track_name' dan 'artist_name'...")
    rows_before_dedup = len(df_processed)
    df_processed.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)
    rows_after_dedup = len(df_processed)
    log_step(f"Duplikat dihapus. {rows_before_dedup - rows_after_dedup} baris duplikat dihapus.")
    log_step(f"Tersisa {rows_after_dedup} baris setelah deduplikasi.")
    log_step("\n--- Data Setelah Deduplikasi (Head) ---")
    print(df_processed.head())

    # Normalisasi fitur numerik
    log_step("Melakukan normalisasi fitur numerik...")
    scaler = StandardScaler()
    df_processed[numeric_features] = scaler.fit_transform(df_processed[numeric_features])
    log_step("Normalisasi selesai.")
    log_step("\n--- Data Setelah Normalisasi (Head) ---")
    # Tampilkan statistik deskriptif setelah normalisasi untuk verifikasi (mean ~0, std ~1)
    print(df_processed[numeric_features].describe())

    log_step(f"Preprocessing selesai. {len(df_processed)} baris siap untuk clustering.")
    return df_processed, numeric_features, scaler

In [28]:
# ============================STEP 3: CLUSTERING============================
def perform_clustering(df_processed, numeric_features, n_clusters=5, random_state=42):
    """
    Melakukan clustering menggunakan K-Means.
    Jumlah cluster dapat disesuaikan.
    """
    print(f"[{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}] STEP 3: Melakukan clustering dengan K-Means ({n_clusters} cluster)...")
    X = df_processed[numeric_features]
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10) # n_init=10 untuk hasil yang lebih stabil
    df_processed['cluster'] = kmeans.fit_predict(X)
    print(f"[{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}] Clustering selesai.")
    return df_processed, kmeans

In [ ]:
# ============================STEP 4: ANALISIS DAN PELABELAN CLUSTER MANUAL============================
def analyze_and_label_clusters(df_processed, numeric_features, kmeans_model, scaler):
    """
    Menganalisis karakteristik setiap cluster dan melakukan pelabelan manual.
    Output: dictionary mapping cluster_id ke label aktivitas, dan statistik cluster.
    """
    log_step("STEP 4: Menganalisis dan melabeli cluster...")

    # Analisis rata-rata fitur per cluster (dalam skala terstandardisasi)
    cluster_centers_scaled = pd.DataFrame(kmeans_model.cluster_centers_, columns=numeric_features)
    cluster_centers_scaled['cluster_id'] = cluster_centers_scaled.index
    log_step("\n--- Rata-rata Fitur (Scaled) per Cluster ---")
    print(cluster_centers_scaled)

    # Inversi transformasi untuk melihat nilai asli (unscaled)
    # Ini sangat membantu untuk interpretasi!
    cluster_centers_unscaled = scaler.inverse_transform(kmeans_model.cluster_centers_)
    cluster_centers_unscaled_df = pd.DataFrame(cluster_centers_unscaled, columns=numeric_features)
    cluster_centers_unscaled_df['cluster_id'] = cluster_centers_unscaled_df.index
    log_step("\n--- Rata-rata Fitur (Unscaled/Asli) per Cluster ---")
    print(cluster_centers_unscaled_df)

    # Analisis distribusi genre per cluster
    log_step("\n--- Distribusi Genre per Cluster (Top 5 Genre per Cluster) ---")
    cluster_genre_distribution = df_processed.groupby('cluster')['genre'].value_counts(normalize=True).unstack(fill_value=0)
    # Tampilkan hanya 5 genre teratas untuk setiap cluster untuk ringkasan
    for cluster_id in cluster_genre_distribution.index:
        top_genres = cluster_genre_distribution.loc[cluster_id].sort_values(ascending=False).head(5)
        log_step(f"Cluster {cluster_id}:")
        print(top_genres)
        print("-" * 20)


    cluster_labels = {
        # Menggunakan np.int64 karena cluster ID dari K-Means adalah tipe NumPy integer
        np.int64(0): 'Upbeat / Dance',
        np.int64(1): 'Olahraga',  # Sesuai rekomendasi sebelumnya, ganti 'Chill'
        np.int64(2): 'Healing',
        np.int64(3): 'Tidur',
        np.int64(4): 'Podcast / Spoken Word' # Sesuai rekomendasi sebelumnya, ganti 'Lainnya_4'
    }

    log_step(f"\n--- Pelabelan Cluster Manual (VERIFIKASI INI!) ---")
    print(cluster_labels)
    log_step("Label ini sudah ditetapkan secara manual berdasarkan analisis rata-rata fitur dan distribusi genre.")
    log_step("Jika Anda ingin mengubahnya, edit dictionary 'cluster_labels' di kode ini.")

    return cluster_labels, cluster_centers_scaled, cluster_genre_distribution

In [30]:
# ============================STEP 5: SIMPAN MODEL DAN METADATA============================
def save_model_and_metadata(kmeans_model, scaler, cluster_labels, numeric_features, df_with_clusters, cluster_centers_unscaled_df, cluster_genre_distribution):
    """
    Menyimpan model yang sudah dilatih, scaler, dan metadata penting lainnya
    ke dalam file .pkl.
    """
    # ... sisa isi fungsi Anda tetap sama
    log_step("STEP 5: Menyimpan model dan metadata ke music_classifier_model.pkl...")

    model_metadata = {
        'kmeans_model': kmeans_model,
        'scaler': scaler,
        'cluster_labels': cluster_labels,
        'numeric_features': numeric_features,
        'df_with_clusters': df_with_clusters, # <--- PASTIKAN INI ADA DI DEFINISI FUNGSI
        'cluster_centers_unscaled_df': cluster_centers_unscaled_df,
        'cluster_genre_distribution': cluster_genre_distribution
    }

    try:
        with open('music_classifier_model.pkl', 'wb') as f:
            pickle.dump(model_metadata, f)
        log_step("Model dan metadata berhasil disimpan ke 'music_classifier_model.pkl'.")
    except Exception as e:
        log_step(f"Error saat menyimpan model: {e}")

In [ ]:
# ============================MAIN EXECUTION FLOW============================
if __name__ == "__main__":
    # Pastikan log_step dan setup logging sudah ada di awal script Anda
    # Contoh:
    # import datetime
    # def log_step(message):
    #     timestamp = datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    #     print(f"{timestamp} {message}")
    #     with open('model_training_log.txt', 'a') as f: # Append ke log file
    #         f.write(f"{timestamp} {message}\n")

    print("=====================================================")
    print("      Personal Music Classifier AI Model Training    ")
    print("      (Dengan Output Detail Setiap Langkah)         ")
    print("=====================================================")
    log_step("Memulai proses pelatihan model...") # Tambahkan log awal

    DATA_FILE = r'D:\_My Data 2\TUGAS BINUS\SMS - 4\4 - Machine Learning\UAP\Progress 6 June 2025\SpotifyFeatures.csv' # Ganti dengan nama file dataset Anda
    N_CLUSTERS = 5 # Jumlah cluster yang diinginkan (sesuaikan!)

    # STEP 1: Load Data
    df = load_data(DATA_FILE)
    if df is None:
        log_step("Gagal memuat data. Menghentikan eksekusi.")
        exit()
    
    # STEP 2: Preprocessing Data
    # Penting: Menggunakan df.copy() untuk menghindari SettingWithCopyWarning
    df_processed, numeric_features, scaler = preprocess_data(df.copy())
    
    # STEP 3: Clustering
    # perform_clustering mengembalikan df_with_clusters (sudah ada kolom 'cluster') dan kmeans_model
    df_with_clusters, kmeans_model = perform_clustering(df_processed.copy(), numeric_features, n_clusters=N_CLUSTERS)
    

    log_step("\n--- Preview Data Lengkap dengan Cluster ID dan Genre ---")
    # Tampilkan beberapa baris pertama dari DataFrame lengkap dengan cluster dan genre
    # Pilih kolom yang relevan untuk dilihat
    print(df_with_clusters[['track_name', 'artist_name', 'genre', 'cluster']].head(10))

    log_step("\n--- Distribusi Genre di Seluruh Dataset Setelah Deduplikasi ---")
    # Menampilkan 10 genre teratas di seluruh dataset setelah preprocessing
    print(df_with_clusters['genre'].value_counts().head(10))

    # Jika Anda ingin menyimpan ini ke file CSV untuk analisis lebih lanjut:
    # df_with_clusters.to_csv("full_dataset_with_clusters_and_genres.csv", index=False)
    # log_step("\nData lengkap dengan cluster ID dan genre disimpan ke 'full_dataset_with_clusters_and_genres.csv'.")

    # STEP 4: Analisis dan Pelabelan Cluster Manual
    # analyze_and_label_clusters mengembalikan cluster_labels, cluster_centers_df_scaled, cluster_genre_distribution
    cluster_labels, cluster_centers_scaled_df, cluster_genre_distribution = analyze_and_label_clusters(
        df_with_clusters, numeric_features, kmeans_model, scaler
    )

    # Menghitung cluster_centers_unscaled_df secara eksplisit
    # karena ini yang diharapkan oleh save_model_and_metadata
    cluster_centers_unscaled = scaler.inverse_transform(kmeans_model.cluster_centers_)
    cluster_centers_unscaled_df = pd.DataFrame(cluster_centers_unscaled, columns=numeric_features)
    cluster_centers_unscaled_df['cluster_id'] = cluster_centers_unscaled_df.index.astype(int) # Pastikan tipe integer

    # Tambahkan mapping aktivitas ke cluster ID ke dalam object cluster_labels
    # Ini akan mempermudah UI untuk mencari cluster berdasarkan aktivitas yang dipilih
    # Pastikan 'activity_to_cluster_id' tidak menimpa cluster ID numerik
    # Menggunakan int(c_id) untuk memastikan tipe Python int, bukan numpy int
    activity_to_cluster_id = {label: int(c_id) for c_id, label in cluster_labels.items() if isinstance(c_id, (int, np.integer))}
    cluster_labels['activity_to_cluster_id'] = activity_to_cluster_id
    
    log_step("\n--- Final Cluster Labels (Termasuk Mapping Aktivitas ke ID Cluster) ---")
    print(cluster_labels)


    # STEP 5: Simpan Model dan Metadata
    # Panggil fungsi save_model_and_metadata dengan urutan parameter yang BENAR
    # Sesuai definisi: save_model_and_metadata(kmeans_model, scaler, cluster_labels, numeric_features, df_with_clusters, cluster_centers_unscaled_df, cluster_genre_distribution)
    save_model_and_metadata(
        kmeans_model=kmeans_model,
        scaler=scaler,
        cluster_labels=cluster_labels,
        numeric_features=numeric_features, # Pastikan ini di posisi yang benar
        df_with_clusters=df_with_clusters, # Ini adalah argumen yang sebelumnya sering jadi masalah
        cluster_centers_unscaled_df=cluster_centers_unscaled_df, # Pastikan ini unscaled
        cluster_genre_distribution=cluster_genre_distribution
    )

    print("\n=====================================================")
    print("        Pelatihan Model Selesai!                       ")
    print("  File 'music_classifier_model.pkl' telah dibuat.    ")
    print("  Sekarang Anda bisa membuat aplikasi Streamlit.      ")
    print("=====================================================")

    # Menampilkan beberapa contoh lagu dari setiap cluster berdasarkan label akhir
    log_step("\nBeberapa contoh lagu dari setiap cluster berdasarkan label akhir:")
    for cluster_name, cluster_id_assigned in cluster_labels['activity_to_cluster_id'].items():
        log_step(f"\n--- Contoh Lagu untuk {cluster_name} (Cluster ID: {cluster_id_assigned}) ---")
        # Filter lagu berdasarkan cluster ID yang sudah dipetakan ke aktivitas
        # Gunakan .sample() dengan try-except atau min() untuk menghindari error jika cluster kosong
        songs_in_cluster = df_with_clusters[df_with_clusters['cluster'] == cluster_id_assigned]
        if not songs_in_cluster.empty:
            # Drop duplicates based on track_name and artist_name before sampling
            sample_songs = songs_in_cluster[['track_name', 'artist_name', 'genre']].drop_duplicates().sample(
                n=min(5, len(songs_in_cluster.drop_duplicates(subset=['track_name', 'artist_name'])))
            )
            for idx, row in sample_songs.iterrows():
                print(f"- {row['track_name']} by {row['artist_name']} (Genre: {row['genre']})")
        else:
            print("Tidak ada lagu ditemukan untuk cluster ini.")

      Personal Music Classifier AI Model Training    
      (Dengan Output Detail Setiap Langkah)         
[2025-06-06 12:41:57] Memulai proses pelatihan model...
[2025-06-06 12:41:57] STEP 1: Memuat data dari jalur yang diberikan: 'D:\_My Data 2\TUGAS BINUS\SMS - 4\4 - Machine Learning\UAP\Progress 6 June 2025\SpotifyFeatures.csv'...
[2025-06-06 12:41:57] Jalur absolut yang dicoba: 'D:\_My Data 2\TUGAS BINUS\SMS - 4\4 - Machine Learning\UAP\Progress 6 June 2025\SpotifyFeatures.csv'
[2025-06-06 12:41:58] Data berhasil dimuat. 232725 baris ditemukan.
[2025-06-06 12:41:58] 
--- Preview Data (Head) ---
   genre        artist_name                        track_name  \
0  Movie     Henri Salvador       C'est beau de faire un Show   
1  Movie  Martin & les fées  Perdu d'avance (par Gad Elmaleh)   
2  Movie    Joseph Williams    Don't Let Me Be Lonely Tonight   
3  Movie     Henri Salvador    Dis-moi Monsieur Gordon Cooper   
4  Movie       Fabien Nataf                         Ouverture   

  

In [1]:
# ============================STEP 0: IMPORT LIBRARY============================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm # Untuk progress bar (opsional tapi bagus untuk data besar)
import datetime # Untuk timestamp di log
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

def log_step(message):
    print(f"[{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {message}")

# --- Fungsi load_data (sudah ada di main.ipynb Anda) ---
def load_data(file_path):
    """
    Memuat dataset dari file CSV.
    Output: DataFrame awal, menampilkan head dan info dasar.
    """
    log_step(f"STEP 1: Memuat data dari jalur yang diberikan: '{file_path}'...")
    try:
        df = pd.read_csv(file_path)
        log_step(f"Data berhasil dimuat. {len(df)} baris ditemukan.")
        print("\n--- Preview Data (Head) ---")
        print(df.head())
        print("\n--- Informasi Data (Info) ---")
        df.info()
        print("\n--- Statistik Deskriptif ---")
        print(df.describe())
        return df
    except FileNotFoundError:
        log_step(f"Error: File tidak ditemukan di '{file_path}'")
        # Coba jalur absolut jika file path tidak bekerja
        abs_file_path = os.path.abspath(file_path)
        log_step(f"Jalur absolut yang dicoba: '{abs_file_path}'")
        try:
            df = pd.read_csv(abs_file_path)
            log_step(f"Data berhasil dimuat dari jalur absolut. {len(df)} baris ditemukan.")
            print(df.head())
            return df
        except Exception as e:
            log_step(f"Gagal memuat data dari jalur absolut: {e}")
            return None
    except Exception as e:
        log_step(f"Error saat memuat data: {e}")
        return None

# --- Fungsi preprocess_data (sudah ada di main.ipynb Anda) ---
def preprocess_data(df):
    """
    Melakukan preprocessing data: pemilihan fitur, penanganan missing values,
    deduplikasi, dan normalisasi fitur numerik.
    Output: DataFrame yang sudah diproses, scaler yang sudah dilatih, dan daftar fitur numerik.
    """
    log_step("STEP 2: Memulai preprocessing data...")
    initial_rows = len(df)
    log_step(f"Baris awal: {initial_rows}")

    # 1. Pemilihan Kolom yang Relevan (sesuaikan jika ada perubahan fitur)
    # Anda perlu daftar fitur numerik di sini untuk scaler dan clustering
    numeric_features = [
        'danceability', 'energy', 'acousticness', 'instrumentalness',
        'liveness', 'valence', 'tempo', 'loudness', 'speechiness'
    ]
    
    # Pastikan juga menyimpan fitur non-numerik yang penting untuk analisis/tampilan
    categorical_features = ['genre', 'artist_name', 'track_name']

    selected_columns = numeric_features + categorical_features
    df_processed = df[selected_columns].copy()
    log_step(f"Memilih {len(selected_columns)} kolom yang relevan.")
    print("\n--- Data Setelah Pemilihan Kolom (Head) ---")
    print(df_processed.head())

    # 2. Penanganan Missing Values (khusus track_name, sisanya numerik)
    log_step("Memeriksa dan mengisi nilai yang hilang...")
    # Untuk fitur numerik, kita bisa pastikan tidak ada yang hilang setelah pemilihan kolom
    # Atau isi dengan median/mean jika ada di data asli (sebelum selection)
    # Namun, umumnya fitur audio Spotify jarang ada missing values numerik
    log_step("Tidak ada nilai hilang ditemukan di fitur numerik.")
    
    # Untuk track_name yang mungkin hilang (ada satu di info Anda sebelumnya)
    # Anda bisa mengisi dengan placeholder atau menghapusnya
    df_processed['track_name'].fillna('Unknown Track', inplace=True)
    
    print("\n--- Data Setelah Penanganan Nilai Hilang (Info) ---")
    df_processed.info()

    # 3. Menghapus Duplikat
    log_step("Menghapus duplikat berdasarkan 'track_name' dan 'artist_name'...")
    initial_dedup_rows = len(df_processed)
    df_processed.drop_duplicates(subset=['track_name', 'artist_name'], inplace=True)
    rows_removed_dedup = initial_dedup_rows - len(df_processed)
    log_step(f"Duplikat dihapus. {rows_removed_dedup} baris duplikat dihapus.")
    log_step(f"Tersisa {len(df_processed)} baris setelah deduplikasi.")
    print("\n--- Data Setelah Deduplikasi (Head) ---")
    print(df_processed.head())

    # 4. Normalisasi Fitur Numerik
    log_step("Melakukan normalisasi fitur numerik...")
    scaler = StandardScaler()
    df_processed[numeric_features] = scaler.fit_transform(df_processed[numeric_features])
    log_step("Normalisasi selesai.")
    print("\n--- Data Setelah Normalisasi (Head) ---")
    print(df_processed[numeric_features].describe()) # Menampilkan statistik deskriptif setelah normalisasi

    log_step(f"Preprocessing selesai. {len(df_processed)} baris siap untuk clustering.")
    return df_processed, scaler, numeric_features

# ============================STEP 3: CLUSTERING DENGAN K-MEANS============================
def perform_clustering(df_processed, numeric_features, n_clusters=5):
    """
    Melakukan clustering K-Means pada data yang sudah diproses (distandardisasi).
    Output: DataFrame dengan kolom 'cluster' dan model K-Means yang sudah dilatih.
    """
    log_step(f"STEP 3: Melakukan clustering dengan K-Means ({n_clusters} cluster)...")

    # Fitur untuk clustering adalah fitur numerik yang sudah diskalakan
    X = df_processed[numeric_features]

    # Inisialisasi dan latih model K-Means
    # n_init='auto' atau nilai numerik seperti 10 disarankan untuk menghindari warning
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # n_init=10 untuk hasil lebih stabil
    
    # Prediksi cluster untuk setiap baris data
    df_processed['cluster'] = kmeans.fit_predict(X)
    
    log_step("Clustering selesai.")

    log_step("\n--- Distribusi Lagu per Cluster ---")
    print(df_processed['cluster'].value_counts())

    log_step("\n--- Head Data dengan Cluster ID ---")
    # Tampilkan beberapa kolom agar mudah diinspeksi
    print(df_processed[['track_name', 'artist_name', 'cluster'] + numeric_features[:2]].head())

    return df_processed, kmeans # Mengembalikan DataFrame dengan cluster ID dan model K-Means

# ============================STEP 4: ANALISIS DAN PELABELAN CLUSTER MANUAL============================
def analyze_and_label_clusters(df_processed, numeric_features, kmeans_model, scaler):
    """
    Menganalisis karakteristik setiap cluster dan melakukan pelabelan manual.
    Output: dictionary mapping cluster_id ke label aktivitas, dan statistik cluster.
    """
    log_step("STEP 4: Menganalisis dan melabeli cluster...")

    # Analisis rata-rata fitur per cluster (dalam skala terstandardisasi)
    cluster_centers_scaled = pd.DataFrame(kmeans_model.cluster_centers_, columns=numeric_features)
    cluster_centers_scaled['cluster_id'] = cluster_centers_scaled.index
    log_step("\n--- Rata-rata Fitur (Scaled) per Cluster ---")
    print(cluster_centers_scaled)

    # Inversi transformasi untuk melihat nilai asli (unscaled)
    cluster_centers_unscaled = scaler.inverse_transform(kmeans_model.cluster_centers_)
    cluster_centers_unscaled_df = pd.DataFrame(cluster_centers_unscaled, columns=numeric_features)
    cluster_centers_unscaled_df['cluster_id'] = cluster_centers_unscaled_df.index
    log_step("\n--- Rata-rata Fitur (Unscaled/Asli) per Cluster ---")
    print(cluster_centers_unscaled_df)

    # Analisis distribusi genre per cluster
    log_step("\n--- Distribusi Genre per Cluster (Top 5 Genre per Cluster) ---")
    # Pastikan kolom 'genre' ada di df_processed
    cluster_genre_distribution = df_processed.groupby('cluster')['genre'].value_counts(normalize=True).unstack(fill_value=0)
    # Tampilkan hanya 5 genre teratas untuk setiap cluster untuk ringkasan
    for cluster_id in cluster_genre_distribution.index:
        top_genres = cluster_genre_distribution.loc[cluster_id].sort_values(ascending=False).head(5)
        log_step(f"Cluster {cluster_id}:")
        print(top_genres)
        print("-" * 20)

    # --- PELABELAN MANUAL CLUSTER DI SINI ---
    # Sesuaikan label ini berdasarkan analisis Anda terhadap cluster_centers_unscaled_df
    # dan cluster_genre_distribution
    cluster_labels = {
        0: 'Instrumental / Cinematic',    # High instrumentalness, Soundtrack/Classical
        1: 'Dance / Groove',              # High danceability, Reggae/Hip-Hop/Dance
        2: 'Upbeat / Alternative',        # This one is actually good ✅
        3: 'Vocal / Dramatic',            # Opera/Movie, low instrumentalness
        4: 'Podcast / Spoken Word'        # This one is perfect ✅
    }

    log_step(f"\n--- Pelabelan Cluster Manual (VERIFIKASI INI!) ---")
    print(cluster_labels)
    log_step("Label ini sudah ditetapkan secara manual berdasarkan analisis rata-rata fitur dan distribusi genre.")
    log_step("Jika Anda ingin mengubahnya, edit dictionary 'cluster_labels' di kode ini.")

    # Kembalikan semua yang dibutuhkan untuk disimpan
    return cluster_labels, cluster_centers_unscaled_df, cluster_genre_distribution

# ============================STEP 5: MENYIMPAN MODEL DAN METADATA============================
def save_model_and_metadata(kmeans_model, scaler, df_with_clusters, numeric_features,
                            cluster_labels, cluster_centers_unscaled_df, cluster_genre_distribution,
                            file_path='music_classifier_model.pkl'):
    """
    Menyimpan model yang sudah dilatih, scaler, dan metadata penting lainnya
    ke dalam satu file pickle.
    """
    log_step(f"STEP 5: Menyimpan model dan metadata ke {file_path}...")

    # Siapkan dictionary untuk metadata
    # Pastikan semua objek yang dibutuhkan aplikasi Streamlit ada di sini
    
    # Membuat activity_to_cluster_id dari cluster_labels (ID -> Nama)
    # activity_to_cluster_id (Nama -> ID)
    activity_to_cluster_id = {v: k for k, v in cluster_labels.items()}

    model_metadata = {
        'kmeans_model': kmeans_model,
        'scaler': scaler,
        'df_with_clusters': df_with_clusters,
        'numeric_features': numeric_features,
        'cluster_centers_unscaled_df': cluster_centers_unscaled_df,
        'cluster_genre_distribution': cluster_genre_distribution,
        'activity_to_cluster_id': activity_to_cluster_id, # Mapping Nama -> ID
        'display_cluster_labels': cluster_labels # Mapping ID -> Nama (sesuai yang Anda definisikan)
    }

    # Simpan ke file pickle
    try:
        with open(file_path, 'wb') as f:
            pickle.dump(model_metadata, f)
        log_step(f"Model dan metadata berhasil disimpan ke '{file_path}'.")
    except Exception as e:
        log_step(f"Error saat menyimpan model: {e}")

# ============================MAIN EXECUTION BLOCK============================
if __name__ == "__main__":
    DATA_PATH = 'SpotifyFeatures.csv' # Ganti dengan jalur file CSV Anda
    N_CLUSTERS = 5 # Jumlah cluster yang diinginkan

    print("=" * 50)
    print("      Personal Music Classifier AI Model Training      ")
    print("      (Dengan Output Detail Setiap Langkah)            ")
    print("=" * 50)
    log_step("Memulai proses pelatihan model...")

    # STEP 1: Memuat Data
    df_raw = load_data(DATA_PATH)
    if df_raw is None:
        log_step("Proses dihentikan karena gagal memuat data.")
    else:
        # STEP 2: Preprocessing Data
        df_processed, scaler, numeric_features = preprocess_data(df_raw.copy())

        # STEP 3: Clustering
        # perform_clustering mengembalikan df_with_clusters (sudah ada kolom 'cluster') dan kmeans_model
        df_with_clusters, kmeans_model = perform_clustering(df_processed.copy(), numeric_features, n_clusters=N_CLUSTERS)

        
        # --- Bagian Baru: Evaluasi Clustering ---
    log_step("\n--- Evaluasi Performa Clustering ---")

    # Ambil data numerik yang digunakan untuk clustering (setelah scaling)
    # Penting: Pastikan ini adalah data yang persis masuk ke K-Means
    X_for_clustering = df_processed[numeric_features]

    # Ambil label cluster dari model K-Means
    cluster_labels_assigned = kmeans_model.labels_

    # Pastikan ada lebih dari satu cluster untuk menghitung metrik ini
    if len(np.unique(cluster_labels_assigned)) > 1:
        try:
            silhouette = silhouette_score(X_for_clustering, cluster_labels_assigned)
            davies_bouldin = davies_bouldin_score(X_for_clustering, cluster_labels_assigned)
            calinski_harabasz = calinski_harabasz_score(X_for_clustering, cluster_labels_assigned)

            log_step(f"Silhouette Score: {silhouette:.4f}")
            log_step(f"Davies-Bouldin Index: {davies_bouldin:.4f}")
            log_step(f"Calinski-Harabasz Index: {calinski_harabasz:.4f}")
        except Exception as e:
            log_step(f"Gagal menghitung metrik clustering: {e}")
            log_step("Ini bisa terjadi jika ada terlalu sedikit sampel atau cluster.")
    else:
        log_step("Tidak dapat menghitung metrik clustering: Hanya satu cluster ditemukan atau data tidak cukup.")

        # --- Akhir Bagian Baru ---

        log_step("\n--- Preview Data Lengkap dengan Cluster ID dan Genre ---")

        # STEP 4: Analisis dan Pelabelan Cluster Manual
        # Mengembalikan cluster_labels (mapping ID ke Nama), cluster_centers_unscaled_df, cluster_genre_distribution
        cluster_labels_map, cluster_centers_unscaled_df, cluster_genre_distribution = \
            analyze_and_label_clusters(df_with_clusters, numeric_features, kmeans_model, scaler)

        # Tambahan untuk log dan contoh lagu dari setiap cluster berdasarkan label akhir
        final_cluster_mapping = {
            'activity_to_cluster_id': {v: k for k, v in cluster_labels_map.items()} # Nama -> ID
        }
        final_cluster_mapping['display_cluster_labels'] = cluster_labels_map # ID -> Nama

        log_step("\n--- Final Cluster Labels (Termasuk Mapping Aktivitas ke ID Cluster) ---")
        print(final_cluster_mapping)

        # Menampilkan contoh lagu dari setiap cluster
        log_step("\nBeberapa contoh lagu dari setiap cluster berdasarkan label akhir:")
        for cluster_id_assigned, cluster_name in cluster_labels_map.items():
            log_step(f"\n--- Contoh Lagu untuk {cluster_name} (Cluster ID: {cluster_id_assigned}) ---")
            # Filter lagu berdasarkan cluster ID yang sudah dipetakan ke aktivitas
            # Gunakan .sample() dengan try-except atau min() untuk menghindari error jika cluster kosong
            songs_in_cluster = df_with_clusters[df_with_clusters['cluster'] == cluster_id_assigned]
            if not songs_in_cluster.empty:
                # Drop duplicates based on track_name and artist_name before sampling
                sample_songs = songs_in_cluster[['track_name', 'artist_name', 'genre']].drop_duplicates().sample(
                    n=min(5, len(songs_in_cluster.drop_duplicates(subset=['track_name', 'artist_name'])))
                )
                for idx, row in sample_songs.iterrows():
                    print(f"- {row['track_name']} by {row['artist_name']} (Genre: {row['genre']})")
            else:
                print("Tidak ada lagu ditemukan untuk cluster ini.")

        # STEP 5: Menyimpan Model dan Metadata
        # Pastikan Anda mengirimkan semua objek yang diperlukan oleh Streamlit
        save_model_and_metadata(
            kmeans_model,
            scaler,
            df_with_clusters,
            numeric_features,
            cluster_labels_map, # Kirim cluster_labels_map yang sudah dari analyze_and_label_clusters
            cluster_centers_unscaled_df,
            cluster_genre_distribution
        )

    print("\n" + "=" * 50)
    print("      Pelatihan Model Selesai!                       ")
    print("      File 'music_classifier_model.pkl' telah dibuat.    ")
    print("      Sekarang Anda bisa membuat aplikasi Streamlit.     ")
    print("=" * 50)

      Personal Music Classifier AI Model Training      
      (Dengan Output Detail Setiap Langkah)            
[2025-06-06 14:25:50] Memulai proses pelatihan model...
[2025-06-06 14:25:50] STEP 1: Memuat data dari jalur yang diberikan: 'SpotifyFeatures.csv'...
[2025-06-06 14:25:51] Data berhasil dimuat. 232725 baris ditemukan.

--- Preview Data (Head) ---
   genre        artist_name                        track_name  \
0  Movie     Henri Salvador       C'est beau de faire un Show   
1  Movie  Martin & les fées  Perdu d'avance (par Gad Elmaleh)   
2  Movie    Joseph Williams    Don't Let Me Be Lonely Tonight   
3  Movie     Henri Salvador    Dis-moi Monsieur Gordon Cooper   
4  Movie       Fabien Nataf                         Ouverture   

                 track_id  popularity  acousticness  danceability  \
0  0BRjO6ga9RKCKjfDqeFgWV           0         0.611         0.389   
1  0BjC1NfoEOOusryehmNudP           1         0.246         0.590   
2  0CoSDzoNIKCRs124s9uTVy           3      

C:\Users\abiyu salam\AppData\Local\Temp\ipykernel_14172\2602287688.py:87: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed['track_name'].fillna('Unknown Track', inplace=True)


       danceability        energy  acousticness  instrumentalness  \
count  1.765140e+05  1.765140e+05  1.765140e+05      1.765140e+05   
mean  -1.468473e-16  4.051181e-16  1.365422e-16     -4.379655e-17   
std    1.000003e+00  1.000003e+00  1.000003e+00      1.000003e+00   
min   -2.542587e+00 -2.019845e+00 -1.102630e+00     -5.331004e-01   
25%   -6.622093e-01 -7.728826e-01 -9.784097e-01     -5.331004e-01   
50%    8.868149e-02  1.261436e-01 -3.163562e-01     -5.328816e-01   
75%    7.450546e-01  8.402894e-01  1.056893e+00     -2.501604e-01   
max    2.351856e+00  1.601562e+00  1.616567e+00      2.559427e+00   

           liveness       valence         tempo      loudness   speechiness  
count  1.765140e+05  1.765140e+05  1.765140e+05  1.765140e+05  1.765140e+05  
mean  -1.655252e-16 -3.265420e-16 -3.501309e-16 -2.833895e-17 -2.189828e-17  
std    1.000003e+00  1.000003e+00  1.000003e+00  1.000003e+00  1.000003e+00  
min   -1.018036e+00 -1.686162e+00 -2.772204e+00 -6.614145e+00 -5.1